In [1]:
%matplotlib inline
from genetic_oscillator_benchmarking_futures_four_core import *

/home/twright/Documents/PhD/Notebooks/notebooks/Genetic Oscillator/genetic_oscillator_benchmarking.py:102: PyGIWarning: Notify was imported without specifying a version first. Use gi.require_version('Notify', '0.7') before import to ensure that the right version gets loaded.
  from gi.repository import Notify


## Properties

In [2]:
P = Atomic(var("x6") > 1)
P

Atomic(x6 > 1)

In [3]:
def prop(k):
    return 125**2*(var("x4") - 0.003)**2 + 3*(var("x6") - 0.5)**2 < k

In [4]:
Qprop = prop(0.032)

In [5]:
Q = Atomic(Qprop); Q

Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032)

In [6]:
I = RIF(3, 3.5)

## Launch Runs

In [7]:
num_runs = 10
duration = 1.5

In [8]:
our_property = P | G(RIF(3, 3.5), Q)

In [9]:
our_property_reversed = G(RIF(0,1), G(RIF(3, 3.5), Q) | P)

In [10]:
def run_name(**kwargs):
    res = 'masked' if kwargs.get('use_masks', False) else 'unmasked'
    if not kwargs.get('tentative_unpreconditioning', True):
        res = f'untentative_{res}' 
    if not kwargs.get('selective_unpreconditioning', True):
        res = f'unselective_{res}' 
    if kwargs.get('symbolic_composition', False):
        res = f'symbolic_{res}' 
    if kwargs.get('unprecondition_upfront', False):
        res = f'upfront_{res}'
    if kwargs.get('reverse', False):
        res = f'{res}_reversed'
    return res

In [11]:
option_sets = [
    dict(use_masks=mask, symbolic_composition=symb,
         unprecondition_upfront=upfront, reverse=reverse)
    for reverse in [False] # [True, False]
    for mask in [True, False]
    for symb in [True, False]
    for upfront in [True, False]
]; option_sets

[{'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': False,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': False,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': True,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': False,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': False,
  'unprecondition_upfront': False,
  'reverse': False}]

In [12]:
# option_sets = [
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'tentative_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': False,
#   'symbolic_composition': False,
#   'unprecondition_upfront': False,
#   'reverse': False},
#  {'use_masks': False,
#   'symbolic_composition': True,
#   'unprecondition_upfront': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': True,
#   'unprecondition_upfront': True,
#   'reverse': False},
# ]
option_sets = [
 {'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False},
]

In [13]:
our_property_reversed

G([0 .. 1], Or([G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032)), Atomic(x6 > 1)]))

In [14]:
option_sets

[{'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False}]

In [15]:
results = {
    (name := run_name(**option_set)): gen_results(
        num_runs,
        our_property_reversed
            if option_set.get('reverse', False)
            else our_property,
        duration,
        task_description=name,
        **option_set,
    )
    for option_set in option_sets
}

/usr/lib64/python3.8/multiprocessing/queues.py:116: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  return _ForkingPickler.loads(res)
/usr/lib64/python3.8/multiprocessing/queues.py:116: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  return _ForkingPickler.loads(res)


phi variables sfs = {x6, x4}


/usr/lib64/python3.8/multiprocessing/queues.py:116: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  return _ForkingPickler.loads(res)


system sfs = System(R, (x1, x2, x3, x4, x5, x6, x7, x8, x9), [[0.97999999999999998 .. 1.0200000000000001], [1.2799999999999998 .. 1.3200000000000001], [0.079999999999999987 .. 0.12000000000000001], [0.079999999999999987 .. 0.12000000000000001], [0.079999999999999987 .. 0.12000000000000001], [1.2799999999999998 .. 1.3200000000000001], [2.4799999999999999 .. 2.5200000000000001], [0.57999999999999996 .. 0.62000000000000011], [1.2799999999999998 .. 1.3200000000000001]], (-0.10000000000000000?*x1*x6 + 50*x3, -x1*x2 + 100*x4, 0.10000000000000000?*x1*x6 - 50*x3, x2*x6 - 100*x4, 0.50000000000000000?*x1 + 5*x3 - 10*x5, -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6, 0.01000000000000000?*x2 + 50*x4 - 0.50000000000000000?*x7, -2*x6*x8 + 0.50000000000000000?*x7 - 0.2000000000000000?*x8 + x9, 2*x6*x8 - x9), varmap=bidict({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6, 'x7': x7, 'x8': x8, 'x9': x9}))
phi variables sfs = {x6, x4}
phi variables sfs = {x6, x4}


In [14]:
results['upfront_symbolic_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 0.50000000000000000], [([0.0000000000000000 .. 0.023769296852601763], True)], mask=Mask([0.0000000000000000 .. 0.50000000000000000], [[0.0000000000000000 .. 0.50000000000000000]]))

In [15]:
results['symbolic_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 0.50000000000000000], [([0.0000000000000000 .. 0.023769296852601763], True)], mask=Mask([0.0000000000000000 .. 0.50000000000000000], [[0.0000000000000000 .. 0.50000000000000000]]))

In [50]:
results

{'upfront_symbolic_masked': [{'prop': G([0 .. 1], Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032))])),
   'duration': 0.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': True,
    'reverse': False},
   'future': <Future at 0x7f6860df09d0 state=finished returned tuple>},
  {'prop': G([0 .. 1], Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032))])),
   'duration': 0.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': True,
    'reverse': False},
   'future': <Future at 0x7f6860ddbaf0 state=finished returned tuple>},
  {'prop': G([0 .. 1], Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032))])),
   'duration': 0.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'u

In [16]:
results_finished = {
    k: v
    for k, v in results.items()
    if all(r['future'].done()
           for r in v)
}; results_finished

{'symbolic_masked': [{'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': False,
    'reverse': False},
   'future': <Future at 0x7fa26e9ead90 state=finished returned tuple>},
  {'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': False,
    'reverse': False},
   'future': <Future at 0x7fa26ea54580 state=finished returned tuple>},
  {'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.003)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': False,
    'reverse':

# Analysis

In [17]:
import pandas as pd

In [18]:
results['symbolic_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 1.0237692968526018], True)], mask=Mask([0.0000000000000000 .. 1.5000000000000000], [[0.0000000000000000 .. 1.5000000000000000]]))

In [19]:
results['symbolic_unmasked'][0]['future'].result()[0]

KeyError: 'symbolic_unmasked'

In [20]:
results['upfront_symbolic_masked'][0]['future'].result()[0]

KeyError: 'upfront_symbolic_masked'

In [21]:
results['symbolic_masked'][0]['future'].result()[2].print()

ran "Precomposing Contexts" 1 times in 0.00021409988403320312
ran "restoring globals" 16 times in 0.0012624263763427734
ran "Running Flow*" 1 times in 93.23690557479858
ran "capturing globals [on exit]" 16 times in 0.0006966590881347656
ran "prepare" 1 times in 0.10110640525817871
ran "toHornerForm" 2 times in 2.86102294921875e-05
ran "mask intersect check" 3336 times in 0.009595155715942383
ran "tentative eval" 870 times in 0.8919317722320557
ran "composing flowpipe" 519 times in 36.13565182685852
ran "pre retrieve" 519 times in 0.00389862060546875
ran "whole domain eval" 519 times in 0.9706556797027588
ran "observable_hf" 517 times in 47.169811487197876
ran "post retrieve" 514 times in 46.734824419021606
ran "root detection" 514 times in 0.006508350372314453
ran "root amalgamation" 514 times in 0.0013668537139892578
ran "prepare in check" 5 times in 9.417533874511719e-05
ran "check boolean [unguarded]" 5 times in 0.46081042289733887
ran "check_single_point" 5 times in 0.4759399890899

In [22]:
results['upfront_symbolic_masked'][0]['future'].result()[2].print()

KeyError: 'upfront_symbolic_masked'

In [23]:
cols = [
    ('Flow*' , 'Running Flow*'),
    ('unpreconditioning', 'unpreconditioning flowpipes'),
    ('P', f'Monitoring atomic {P}'),
    ('Q',  f'Monitoring atomic {Q}'),
    ('G(P | G(Q))', [
        f'Monitoring Signal for {our_property}',
        f'Monitoring Signal for {our_property_reversed}',
    ]),
    ('tentative eval', 'tentative eval'),
    ('eval', 'whole domain eval'),
    ('root detection', 'root detection'),
    ('composing flowpipe', 'composing flowpipe'),
    ('composing atomic', 'post retrieve'),
]
cols_short, cols_full = zip(*cols)

## Raw results

In [24]:
total_times(
    results['masked'],
    cols_full,
    cols_short,
)

KeyError: 'masked'

In [25]:
total_times(
    results['upfront_masked'],
    cols_full,
    cols_short,
)

KeyError: 'upfront_masked'

In [26]:
_['Flow*'].std()

TypeError: 'Signal' object is not subscriptable

## Export Data

In [27]:
import pickle

In [28]:
total_timess = {
    s: total_times(res, cols_full, cols_short)
    for s, res in results.items()
}

In [30]:
for s, res in results_finished.items():
    with open(f'data/results_doubles_{s}.pkl', 'wb') as file:
        pickle.dump([f.result() for f in futures(res)],
                    file)

In [31]:
for s, d in total_timess.items():
    d.to_csv(f'data/benchmark_doubles_{s}.csv', index_label=False)

In [32]:
with open('data/benchmark_doubles_metadata.pkl', 'wb') as file:
    pickle.dump((cols_full, cols_short, list(results_finished.keys())), file)